In [ ]:
import xarray as xr
import numpy as np
import rioxarray
import h5netcdf
import h5py
import netCDF4
import scipy.ndimage

import holoviews as hv

import hvplot.xarray
import geoviews as gv

import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import matplotlib as mpl

## This notebook is associated with Appendix A of the paper

It produces the surface velocity divergence figure in the appendix and several other plots (not included in the manuscript) that can be used to build intuition around the stability of the ODE in Greenland based on available data.

In [ ]:
surface_velocity = xr.open_dataset('data/GRE_G0120_0000.nc')
surface_velocity_spacing_m = np.median(np.diff(surface_velocity['x']))
assert(np.max(np.diff(surface_velocity['x'])) == surface_velocity_spacing_m)
assert(np.min(np.diff(surface_velocity['x'])) == surface_velocity_spacing_m)

age_model = xr.open_dataset('data/RRRAG4_Greenland_1993_2013_01_age_grid.nc',
                               engine='h5netcdf').rename(
                                   {'number of grid points in x-direction': 'x', 'number of grid points in y-direction': 'y', 'number of vertical layers': 'layers', 'number of isochrones': 'original_isochrones',
                                    'x': 'x_km', 'y': 'y_km'}
                                   )
# Convert km to meters for consistency
age_model = age_model.assign_coords({'x':age_model.x_km[:,0]*1000, 'y':age_model.y_km[0,:]*1000})

bedmachine = xr.open_dataset('data/BedMachineGreenland-v5.nc')

In [ ]:
crs_3413 = ccrs.Stereographic(central_latitude=90, central_longitude=-45, true_scale_latitude=70)

In [ ]:
isochrone_ages = np.logspace(3, 6, 20)

def interpolate_age_depths(age_norm, age_norm_depths = age_model['depth_norm'].to_numpy(), query_ages = isochrone_ages):
    return np.interp(query_ages, age_norm, age_norm_depths, left=np.nan, right=np.nan)

isochrones_norm = xr.apply_ufunc(interpolate_age_depths, age_model['age_norm'], input_core_dims=[['layers']], output_core_dims=[['isochrone']], vectorize=True)
isochrones_norm = isochrones_norm.assign_coords({'isochrone': isochrone_ages})

In [ ]:
smoothing_kernel_std = 5000 # m

In [ ]:
# isochrones_norm is in normalized depth (0 == surface, 1 == bed)
# want to convert this into elevation relative to the geoid in order to find layer geometry and then slope
layer_elevation = ((1 - isochrones_norm) * age_model['thick']) + bedmachine['bed'].interp(x=age_model['x'], y=age_model['y'])
layer_elevation.name = "Layer Elevation"

layer_elevation_spacing_m = np.median(np.diff(layer_elevation['x']))
assert(np.max(np.diff(layer_elevation['x'])) == layer_elevation_spacing_m)
assert(np.min(np.diff(layer_elevation['x'])) == layer_elevation_spacing_m)
assert(np.max(np.diff(layer_elevation['y'])) == layer_elevation_spacing_m)
assert(np.min(np.diff(layer_elevation['y'])) == layer_elevation_spacing_m)

layer_elevation_filt = xr.apply_ufunc(scipy.ndimage.gaussian_filter, layer_elevation, input_core_dims=[['x', 'y']], output_core_dims=[['x', 'y']], vectorize=True, kwargs={'sigma': smoothing_kernel_std / layer_elevation_spacing_m})

In [ ]:
surf_u_filt = xr.apply_ufunc(scipy.ndimage.gaussian_filter, surface_velocity['vx'], kwargs={'sigma': smoothing_kernel_std / surface_velocity_spacing_m, 'truncate': 2})
surf_v_filt = xr.apply_ufunc(scipy.ndimage.gaussian_filter, surface_velocity['vx'], kwargs={'sigma': smoothing_kernel_std / surface_velocity_spacing_m, 'truncate': 2})

In [ ]:
layer_dx_dz = layer_elevation_filt.differentiate("x").differentiate("isochrone") / layer_elevation.differentiate("isochrone")
layer_dy_dz = layer_elevation_filt.differentiate("y").differentiate("isochrone") / layer_elevation.differentiate("isochrone")

us = surf_u_filt.interp(x=age_model['x'], y=age_model['y']).squeeze() # Surface velocity X component
vs = surf_v_filt.interp(x=age_model['x'], y=age_model['y']).squeeze() # Surface velocity Y component
us_dx = us.differentiate("x")
vs_dy = vs.differentiate("y")

#stab_crit = (us * layer_dx_dz) + (vs * layer_dy_dz)
#stab_crit = us_dx + vs_dy
stab_crit = (us * layer_dx_dz) + (vs * layer_dy_dz) + us_dx + vs_dy

p = stab_crit.hvplot.image('x', 'y', crs=crs_3413, coastline=True, cmap='bwr', frame_height=500, clim=(-1e-3,1e-3), rasterize=False, dynamic=True)

p

In [ ]:
surf_vel_divergence = us_dx + vs_dy
p = surf_vel_divergence.hvplot.image('x', 'y', crs=crs_3413, coastline=True, cmap='bwr', frame_height=500, clim=(-1e-3,1e-3), rasterize=False, clabel='Surface Velocity Divergence [1/year]')
hvplot.save(p, f'surf_vel_divergence_smoothstd{smoothing_kernel_std}.png')
p

In [ ]:
layer_slopes = np.sqrt( (layer_elevation_filt.differentiate("x") ** 2) + (layer_elevation_filt.differentiate("y") ** 2) )
p = (layer_slopes * 1000).hvplot.image('x', 'y', crs=crs_3413, coastline=True, frame_height=500, clim=(0,20), rasterize=False)
p.opts(title=f"Layer slope [m/km]")

In [ ]:
qts = np.arange(0.1, 0.9, 0.01)

layer_geom_terms = (us * layer_dx_dz) + (vs * layer_dy_dz)
surf_vel_divergence = us_dx + vs_dy

p = layer_geom_terms.quantile(qts).rename('(us * layer_dx_dz) + (vs * layer_dy_dz)').hvplot(label='(us * layer_dx_dz) + (vs * layer_dy_dz)') * \
    surf_vel_divergence.quantile(qts).rename('us_dx + vs_dy').hvplot(label='us_dx + vs_dy').opts(xlabel='') * \
    (layer_geom_terms + surf_vel_divergence).quantile(qts).rename('sum').hvplot(label='sum').opts(xlabel='')
p.opts(ylabel='', xlabel='quantile', legend_position="top_left", show_grid=True, ylim=(-5e-4, 1e-3), title=f'Smoothing Std {smoothing_kernel_std/1000} km')
#hvplot.save(p, f'stability_terms_comparison_smoothstd{smoothing_kernel_std}.png')
p

In [ ]:
abs_ratio = np.abs(surf_vel_divergence / layer_geom_terms)
p = abs_ratio.quantile(qts).rename('quantiles').hvplot(label='abs(us_dx + vs_dy) / abs((us * layer_dx_dz) + (vs * layer_dy_dz))').opts(xlabel='')
p.opts(ylabel='Ratio of surface velocity divergence to\nlayer geometry terms', xlabel='quantile', show_grid=True, ylim=(0, 10))
#p.opts(ylabel='', xlabel='quantile', legend_position="top_left", show_grid=True, ylim=(0, 5), title=f'Smoothing Std {smoothing_kernel_std/1000} km')

In [ ]:
all_ratios = abs_ratio.to_numpy().flatten()
all_ratios = all_ratios[~np.isnan(all_ratios)]
len(all_ratios[all_ratios > 2]) / len(all_ratios)